In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
%run ../conf.py
%run ../run_xgboost.py
%run ../data.py

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 2500)

In [3]:
cvdata = get_cvdata()
msg('Loaded cvdata: {}'.format(cvdata))
xtrain, xtest = cvdata.xtrain, cvdata.xtest
ytrain, ytest = cvdata.ytrain, cvdata.ytest
featnames = cvdata.featnames

18-00:56:29: no cvdata file /Users/aa/lab/kaggle/springleaf/data/cvdata.pkl. will load data and gen feats
18-00:56:29: loading data
18-00:56:29:     reading from pickle file /Users/aa/lab/kaggle/springleaf/data/train.pkl
18-00:56:30:     update_df_with_pvi_pkldicts
18-00:56:30:     completed update_df_with_pvi_pkldicts
18-00:56:30: train_test_split
18-00:56:30: xtrain: (750, 1996), xtest: (250, 1996), ytrain: (750,), ytest: (250,)
18-00:56:30: dropping 12 low relevance cols
18-00:56:30: gen_text_feats_from_title_columns (2 cols will be dropped, text feats will be merged later)
18-00:56:30: gen_fancy_feats
18-00:56:30:     adding feat: zipcode feats
18-00:56:30:     adding feat: same state
18-00:56:30:     adding feat: date ordinal diffs
18-00:56:33: encode_categorical
18-00:56:33:     size before encoding = (750, 2106), (250, 2106)
18-00:56:33:     size after encoding = (750, 2639), (250, 2639)
18-00:56:33: merge_text_feats
18-00:56:33:     after merge_text_feats: xtrain: (750, 2643), 

In [ ]:
xtrain[:, 1975], xtrain[:, 1974]

In [ ]:
from scipy.stats import pearsonr
pearsonr(xtrain[:, 1975].toarray(), xtrain[:, 210].toarray())

In [7]:
xtrain.shape

(750, 2590)

In [ ]:
import itertools
from scipy.stats import pearsonr

removedcols = set()
cols = range(10)
for i, comb in enumerate(itertools.combinations(cols, 2)):
    if i%100==0:
        print(i)
    c0, c1 = comb[0], comb[1]
    if (c0 not in removedcols) and (c1 not in removedcols):
        corr, pval = pearsonr(xtrain[:, c0].toarray(), xtrain[:, c1].toarray())
        if isinstance(corr, np.ndarray):
            corr = corr[0]
        if isinstance(pval, np.ndarray):
            pval = pval[0]

        #print('CORR_RPT, {}, {}, {}, {}'.format(c0, c1, corr, pval))
        if (abs(corr) > 0.7) and (pval<0.01):
            print('({}, {}): removing {}. corr={}, pval={}, colsremoved={}'.format(c0, c1, c1, corr, pval, len(removedcols)))
            removedcols.add(c1)

print('Removed cols', removedcols)

In [ ]:
import itertools
from scipy.stats import pearsonr

removedcols = set()
cols = range(4)
for comb in itertools.combinations(cols, 2):
    c0, c1 = comb[0], comb[1]
    print(c0, c1)
    if (c0 not in removedcols) and (c1 not in removedcols):
        corr, pval = pearsonr(xtrain[:, c0].toarray(), xtrain[:, c1].toarray())
        print(corr, pval)

In [ ]:
x = xtrain[:, [0,1,2,3,4]]

In [ ]:
from scipy.stats import pearsonr
pearsonr(xtrain[:, 0].toarray(), xtrain[:, 2].toarray())

In [ ]:
corr = np.corrcoef(xtrain[:, 0:10].toarray().T)
removedcols = set()
#print(corr.shape)
for i, r in enumerate(corr):
    print('****', i)
    for j, c in enumerate(r):
        cij = corr[i, j]
        print('CORR_RPT, {}, {}, {}'.format(i, j, cij))
        if (i != j) and (abs(cij) > 0.7):
            if (i not in removedcols) and (j not in removedcols):
                print('Removing', j)
                removedcols.add(j)
            
print(removedcols)
print('Removed', len(removedcols))

In [13]:
remcols = {4, 6, 7, 9, 11, 12, 13, 14, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 74, 76, 77, 78, 79, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 107, 108, 109, 110, 111, 112, 113, 114, 117, 118, 120, 121, 123, 125, 126, 128, 130, 132, 133, 134, 137, 138, 139, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 170, 171, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 192, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 215, 218, 219, 221, 222, 223, 226, 227, 228, 230, 231, 232, 235, 237, 238, 241, 243, 244, 245, 246, 247, 248, 249, 250, 252, 253, 255, 256, 257, 262, 263, 265, 266, 267, 268, 269, 271, 273, 274, 275, 279, 280, 281, 283, 284, 285, 286, 290, 291, 292, 293, 297, 298, 306, 307, 310, 311, 313, 314, 315, 316, 317, 318, 319, 323, 324, 335, 338, 339, 340, 342, 343, 345, 349, 351, 352, 353, 355, 356, 357, 358, 360, 361, 364, 374, 376, 380, 381, 383, 384, 385, 386, 388, 392, 393, 395, 398, 402, 406, 408, 410, 412, 413, 415, 418, 421, 422, 423, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 445, 446, 447, 448, 450, 451, 452, 455, 456, 457, 458, 459, 462, 464, 466, 467, 469, 472, 473, 480, 481, 488, 492, 493, 494, 497, 499, 500, 501, 503, 505, 506, 508, 510, 511, 512, 517, 518, 519, 520, 521, 525, 526, 527, 529, 530, 533, 536, 538, 542, 543, 549, 552, 555, 557, 558, 559, 560, 561, 563, 564, 565, 566, 567, 568, 569, 572, 573, 574, 577, 579, 580, 583, 585, 589, 590, 591, 593, 596, 599, 603, 607, 608, 611, 612, 613, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 629, 630, 631, 632, 633, 634, 635, 636, 638, 639, 640, 641, 642, 643, 644, 645, 646, 648, 653, 654, 655, 656, 657, 664, 665, 666, 669, 670, 671, 672, 673, 674, 675, 677, 678, 679, 680, 682, 684, 685, 686, 688, 689, 690, 691, 693, 695, 696, 697, 698, 701, 702, 704, 705, 706, 707, 708, 709, 710, 711, 712, 714, 716, 717, 718, 721, 723, 724, 725, 727, 728, 729, 731, 733, 736, 742, 743, 744, 745, 747, 748, 750, 751, 752, 754, 755, 756, 757, 758, 761, 766, 767, 768, 769, 770, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 786, 787, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 821, 823, 824, 827, 830, 831, 832, 833, 834, 835, 836, 838, 839, 840, 841, 842, 843, 844, 846, 847, 848, 852, 858, 861, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 882, 884, 886, 887, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 906, 907, 908, 909, 910, 911, 912, 915, 916, 917, 918, 919, 920, 922, 923, 925, 926, 928, 931, 932, 934, 935, 936, 939, 940, 941, 944, 946, 947, 948, 949, 950, 953, 955, 956, 958, 959, 960, 961, 962, 963, 964, 966, 968, 969, 971, 972, 974, 976, 979, 980, 981, 985, 986, 987, 988, 991, 993, 995, 997, 1001, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1127, 1128, 1131, 1135, 1136, 1137, 1138, 1142, 1144, 1145, 1146, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1161, 1163, 1164, 1167, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1190, 1191, 1194, 1195, 1196, 1198, 1199, 1200, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1238, 1239, 1240, 1241, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1285, 1287, 1289, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1416, 1417, 1419, 1420, 1422, 1423, 1424, 1425, 1426, 1428, 1429, 1431, 1432, 1434, 1435, 1436, 1437, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1473, 1474, 1475, 1476, 1477, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1532, 1535, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1556, 1557, 1558, 1561, 1562, 1563, 1566, 1567, 1571, 1572, 1575, 1576, 1577, 1580, 1581, 1582, 1585, 1586, 1587, 1588, 1592, 1593, 1596, 1597, 1598, 1599, 1602, 1604, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1677, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729, 1735, 1736, 1737, 1739, 1741, 1743, 1744, 1745, 1746, 1747, 1748, 1750, 1751, 1752, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1764, 1765, 1766, 1767, 1769, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1780, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1846, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1892, 1893, 1895, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1940, 1941, 1942, 1943, 1944, 1946, 1947, 1950, 1951, 1952, 1954, 1955, 1957, 1958, 1960, 1963, 1964, 1965, 1966, 1967, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1991, 1992, 1993, 1994, 1996, 1997, 1998, 1999, 2001, 2002, 2003, 2004, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2085, 2087, 2090, 2135, 2136, 2138, 2141, 2153, 2154, 2156, 2157, 2163, 2166, 2174, 2177, 2182, 2185, 2190, 2193, 2194, 2199, 2201, 2202, 2204, 2210, 2220, 2221, 2271, 2272, 2273, 2274, 2276, 2277, 2279, 2281, 2286, 2287, 2288, 2289, 2290, 2292, 2296, 2299, 2397, 2398, 2731, 2957, 3057, 3083, 3084, 3085, 3089, 3090, 3091, 3093, 3098, 3104, 3111, 3117, 3118, 3125, 3130, 3131, 3133, 3135, 3137, 3141, 3144, 3147, 3151, 3152, 3153, 3154, 3156, 3157, 3159, 3163, 3165, 3171, 3172, 3174, 3181, 3183, 3195, 3198, 3199, 3202, 3208, 3211, 3213, 3216, 3237, 3238, 3239, 3240, 3241, 3244, 3245, 3247, 3249, 3255, 3257, 3258, 3259, 3263, 3264, 3266, 3268, 3271, 3273, 3274, 3278, 3284, 3289, 3290, 3292, 3293, 3295, 3296, 3303, 3304, 3308, 3312, 3313, 3315, 3317, 3318, 3320, 3322, 3324, 3326, 3333, 3336, 3337, 3338, 3340, 3342, 3346, 3347, 3352, 3353, 3354, 3355, 3358, 3361, 3362, 3364, 3366, 3367, 3368, 3369, 3371, 3377, 3378, 3380, 3381, 3382, 3383, 3384, 3385, 3387, 3390, 3392, 3393, 3394, 3404, 3405, 3406, 3407, 3411, 3416, 3419, 3420, 3423, 3424, 3425, 3426, 3435, 3439, 3440, 3441, 3442, 3443, 3445, 3451, 3455, 3459, 3461, 3462, 3463, 3464, 3468, 3469, 3470, 3471, 3472, 3473, 3476, 3477, 3480, 3481, 3482, 3484, 3485, 3486, 3489, 3490, 3491, 3492, 3493, 3494, 3495, 3496, 3498, 3499, 3503, 3506, 3508, 3511, 3514, 3515, 3516, 3519, 3520, 3522, 3523, 3524, 3526, 3527, 3528, 3529, 3534, 3538, 3539, 3540, 3543, 3544, 3545, 3547, 3549, 3551, 3553, 3554, 3555, 3556, 3557, 3560, 3562, 3563, 3564, 3565, 3567, 3569, 3570, 3571, 3574}

In [14]:
print(len(remcols))

2590-len(remcols)

1857


733

In [ ]:
for i in range(1940, 2000):
    if i not in remcols:
        print(i)